In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.nn as dglnn
import dgl
import random
from sklearn.metrics import f1_score

from omega.models import create_model

In [2]:
def build_example_graph(
    num_complete_nodes,
    num_connecting_nodes,
    num_infer_nodes,
    num_classes,
    num_inputs):



    src_ids = []
    dst_ids = []

    for i in range(num_complete_nodes):
        src_ids += [i] * num_complete_nodes
    dst_ids = [i for i in range(num_complete_nodes)] * num_complete_nodes
    
    for c in range(num_connecting_nodes):
        connecting_node_id = num_complete_nodes + c

        src_ids.append(num_complete_nodes - 1)
        dst_ids.append(connecting_node_id)
        src_ids.append(connecting_node_id)
        dst_ids.append(num_complete_nodes - 1)
        src_ids.append(connecting_node_id)
        dst_ids.append(connecting_node_id)

    for i in range(num_infer_nodes):
        for c in range(num_connecting_nodes):
            connecting_node_id = num_complete_nodes + c
            infer_node_id = num_complete_nodes + num_connecting_nodes + i

            src_ids.append(connecting_node_id)
            dst_ids.append(infer_node_id)
            src_ids.append(infer_node_id)
            dst_ids.append(connecting_node_id)
        src_ids.append(infer_node_id)
        dst_ids.append(infer_node_id)

    g = dgl.graph((src_ids, dst_ids))
    train_nids = [i for i in range(num_complete_nodes + num_connecting_nodes)]
    infer_nids = [i + num_complete_nodes + num_connecting_nodes for i in range(num_infer_nodes)]

    num_nodes = num_complete_nodes + num_connecting_nodes + num_infer_nodes
    g.ndata["features"] = torch.rand(num_nodes, num_inputs)
    g.ndata["labels"] = torch.tensor([random.randint(0, num_classes-1) for _ in range(num_nodes)])

    return g, train_nids, infer_nids

def cal_acc(logits, labels):
    y_pred = np.argmax(logits, axis=1)
    return (labels == y_pred).astype("float64").mean()

In [3]:
random_seed = 940321
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

device = "cuda:0"
num_layers = 3
num_complete_nodes = 1024
num_connecting_nodes = 1
num_infer_nodes = 1024
num_classes = 64
num_inputs = 64
num_hiddens = 64

model = create_model(
    "sage",
    num_inputs,
    num_hiddens,
    num_classes,
    num_layers,
    gat_heads=[],
    dropout=0.5
).to(device)

g, train_nids, infer_nids = build_example_graph(
    num_complete_nodes,
    num_connecting_nodes,
    num_infer_nodes,
    num_classes,
    num_inputs)
g = g.to(device)

In [4]:
num_epochs = 30000

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(num_epochs):
    model.train()

    logits = model([g] * num_layers, g.ndata["features"])
    labels = g.ndata["labels"]
    loss = loss_fn(logits, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        model.eval()
        logits = model([g] * num_layers, g.ndata["features"])
        acc = cal_acc(logits.cpu().numpy(), g.ndata["labels"].cpu().numpy())
        print(f"Epoch={epoch} Loss={loss.item()} Acc={acc}")


Epoch=0 Loss=7.254260540008545 Acc=0.016593460224499756
Epoch=1 Loss=6.7067670822143555 Acc=0.01708150317227916
Epoch=2 Loss=6.171792984008789 Acc=0.01708150317227916
Epoch=3 Loss=6.039567470550537 Acc=0.015617374328940947
Epoch=4 Loss=5.793638706207275 Acc=0.01903367496339678
Epoch=5 Loss=5.6246232986450195 Acc=0.020985846754514398
Epoch=6 Loss=5.282201290130615 Acc=0.021473889702293802
Epoch=7 Loss=5.08815860748291 Acc=0.020985846754514398
Epoch=8 Loss=4.998082160949707 Acc=0.020985846754514398
Epoch=9 Loss=4.925275802612305 Acc=0.01903367496339678
Epoch=10 Loss=4.852590084075928 Acc=0.018545632015617375
Epoch=11 Loss=4.688154220581055 Acc=0.017569546120058566
Epoch=12 Loss=4.658957481384277 Acc=0.016593460224499756
Epoch=13 Loss=4.572160720825195 Acc=0.016105417276720352
Epoch=14 Loss=4.540975570678711 Acc=0.017569546120058566
Epoch=15 Loss=4.459570407867432 Acc=0.01708150317227916
Epoch=16 Loss=4.498503684997559 Acc=0.016593460224499756
Epoch=17 Loss=4.407596588134766 Acc=0.0170815

Epoch=172 Loss=4.1340227127075195 Acc=0.03611517813567594
Epoch=173 Loss=4.133650302886963 Acc=0.03465104929233773
Epoch=174 Loss=4.132486343383789 Acc=0.03465104929233773
Epoch=175 Loss=4.130100250244141 Acc=0.03465104929233773
Epoch=176 Loss=4.132017612457275 Acc=0.035627135187896536
Epoch=177 Loss=4.133258819580078 Acc=0.035627135187896536
Epoch=178 Loss=4.123678207397461 Acc=0.036603221083455345
Epoch=179 Loss=4.133424282073975 Acc=0.03611517813567594
Epoch=180 Loss=4.131444931030273 Acc=0.035627135187896536
Epoch=181 Loss=4.132372856140137 Acc=0.035627135187896536
Epoch=182 Loss=4.133312225341797 Acc=0.035627135187896536
Epoch=183 Loss=4.12958288192749 Acc=0.03513909224011713
Epoch=184 Loss=4.131330490112305 Acc=0.03513909224011713
Epoch=185 Loss=4.12456750869751 Acc=0.035627135187896536
Epoch=186 Loss=4.128727912902832 Acc=0.03513909224011713
Epoch=187 Loss=4.124911785125732 Acc=0.03513909224011713
Epoch=188 Loss=4.1357293128967285 Acc=0.03465104929233773
Epoch=189 Loss=4.1298861

Epoch=317 Loss=4.095611095428467 Acc=0.04441190824792582
Epoch=318 Loss=4.092074394226074 Acc=0.04441190824792582
Epoch=319 Loss=4.087752342224121 Acc=0.04441190824792582
Epoch=320 Loss=4.0918660163879395 Acc=0.04441190824792582
Epoch=321 Loss=4.094906806945801 Acc=0.043923865300146414
Epoch=322 Loss=4.101631164550781 Acc=0.04343582235236701
Epoch=323 Loss=4.086675643920898 Acc=0.042947779404587605
Epoch=324 Loss=4.088594913482666 Acc=0.0424597364568082
Epoch=325 Loss=4.090119361877441 Acc=0.042947779404587605
Epoch=326 Loss=4.094536304473877 Acc=0.042947779404587605
Epoch=327 Loss=4.081902027130127 Acc=0.042947779404587605
Epoch=328 Loss=4.09175443649292 Acc=0.04489995119570522
Epoch=329 Loss=4.090457439422607 Acc=0.04685212298682284
Epoch=330 Loss=4.086857318878174 Acc=0.04685212298682284
Epoch=331 Loss=4.096202373504639 Acc=0.04685212298682284
Epoch=332 Loss=4.092041015625 Acc=0.04636408003904344
Epoch=333 Loss=4.081801414489746 Acc=0.04587603709126403
Epoch=334 Loss=4.0861501693725

Epoch=488 Loss=4.008485794067383 Acc=0.06832601268911664
Epoch=489 Loss=4.018851280212402 Acc=0.06930209858467545
Epoch=490 Loss=4.003722667694092 Acc=0.07174231332357248
Epoch=491 Loss=4.027460098266602 Acc=0.07223035627135188
Epoch=492 Loss=4.01742696762085 Acc=0.0741825280624695
Epoch=493 Loss=4.017922401428223 Acc=0.07271839921913129
Epoch=494 Loss=4.014907360076904 Acc=0.07320644216691069
Epoch=495 Loss=4.008220672607422 Acc=0.07223035627135188
Epoch=496 Loss=4.016304969787598 Acc=0.07223035627135188
Epoch=497 Loss=4.031622886657715 Acc=0.0741825280624695
Epoch=498 Loss=4.01268196105957 Acc=0.07320644216691069
Epoch=499 Loss=4.018813610076904 Acc=0.0736944851146901
Epoch=500 Loss=4.01631498336792 Acc=0.0736944851146901
Epoch=501 Loss=4.027962684631348 Acc=0.07613469985358712
Epoch=502 Loss=4.018263339996338 Acc=0.0736944851146901
Epoch=503 Loss=4.000802040100098 Acc=0.0736944851146901
Epoch=504 Loss=4.010254383087158 Acc=0.07759882869692533
Epoch=505 Loss=4.0073747634887695 Acc=0.

Epoch=656 Loss=3.9421772956848145 Acc=0.11859443631039532
Epoch=657 Loss=3.954233169555664 Acc=0.12201073694485115
Epoch=658 Loss=3.9346871376037598 Acc=0.12445095168374817
Epoch=659 Loss=3.942436456680298 Acc=0.12542703757930698
Epoch=660 Loss=3.923448324203491 Acc=0.12396290873596877
Epoch=661 Loss=3.9437179565429688 Acc=0.12493899463152758
Epoch=662 Loss=3.9308648109436035 Acc=0.12445095168374817
Epoch=663 Loss=3.9315853118896484 Acc=0.12493899463152758
Epoch=664 Loss=3.9300615787506104 Acc=0.127867252318204
Epoch=665 Loss=3.9386065006256104 Acc=0.1273792093704246
Epoch=666 Loss=3.9295125007629395 Acc=0.13079551000488043
Epoch=667 Loss=3.9211597442626953 Acc=0.12981942410932162
Epoch=668 Loss=3.9311487674713135 Acc=0.12933138116154222
Epoch=669 Loss=3.9260952472686768 Acc=0.12542703757930698
Epoch=670 Loss=3.944525718688965 Acc=0.12493899463152758
Epoch=671 Loss=3.943398952484131 Acc=0.12445095168374817
Epoch=672 Loss=3.942495584487915 Acc=0.1273792093704246
Epoch=673 Loss=3.9300041

...

Epoch=29192 Loss=2.435394048690796 Acc=0.7242557345046364
Epoch=29193 Loss=2.4383277893066406 Acc=0.7320644216691069
Epoch=29194 Loss=2.494150161743164 Acc=0.7349926793557833
Epoch=29195 Loss=2.459868907928467 Acc=0.744265495363592
Epoch=29196 Loss=2.5284435749053955 Acc=0.7447535383113714
Epoch=29197 Loss=2.5170791149139404 Acc=0.7408491947291361
Epoch=29198 Loss=2.4106855392456055 Acc=0.7364568081991215
Epoch=29199 Loss=2.4698784351348877 Acc=0.7306002928257687
Epoch=29200 Loss=2.5080103874206543 Acc=0.7218155197657394
Epoch=29201 Loss=2.486212968826294 Acc=0.7198633479746218
Epoch=29202 Loss=2.494965076446533 Acc=0.7218155197657394
Epoch=29203 Loss=2.516792058944702 Acc=0.7227916056612982
Epoch=29204 Loss=2.5195212364196777 Acc=0.7291361639824304
Epoch=29205 Loss=2.46706485748291 Acc=0.7291361639824304
Epoch=29206 Loss=2.4683709144592285 Acc=0.7325524646168863
Epoch=29207 Loss=2.476043939590454 Acc=0.7379209370424598
Epoch=29208 Loss=2.475048542022705 Acc=0.7437774524158126
Epoch=29

Epoch=29360 Loss=2.4575014114379883 Acc=0.7354807223035628
Epoch=29361 Loss=2.451009750366211 Acc=0.7398731088335774
Epoch=29362 Loss=2.433891534805298 Acc=0.7462176671547096
Epoch=29363 Loss=2.451465368270874 Acc=0.7462176671547096
Epoch=29364 Loss=2.423501968383789 Acc=0.7388970229380185
Epoch=29365 Loss=2.4819600582122803 Acc=0.7413372376769156
Epoch=29366 Loss=2.52272891998291 Acc=0.7384089799902391
Epoch=29367 Loss=2.4941654205322266 Acc=0.7369448511469009
Epoch=29368 Loss=2.4760894775390625 Acc=0.7354807223035628
Epoch=29369 Loss=2.458214521408081 Acc=0.7301122498779893
Epoch=29370 Loss=2.443859100341797 Acc=0.731088335773548
Epoch=29371 Loss=2.4868035316467285 Acc=0.7345046364080039
Epoch=29372 Loss=2.4796488285064697 Acc=0.739385065885798
Epoch=29373 Loss=2.4596192836761475 Acc=0.7349926793557833
Epoch=29374 Loss=2.4653875827789307 Acc=0.7281600780868717
Epoch=29375 Loss=2.463343381881714 Acc=0.7208394338701806
Epoch=29376 Loss=2.5181021690368652 Acc=0.7115666178623719
Epoch=29

Epoch=29528 Loss=2.4373409748077393 Acc=0.7335285505124451
Epoch=29529 Loss=2.4369750022888184 Acc=0.7413372376769156
Epoch=29530 Loss=2.5274884700775146 Acc=0.7457296242069302
Epoch=29531 Loss=2.435845375061035 Acc=0.7481698389458272
Epoch=29532 Loss=2.463834762573242 Acc=0.744265495363592
Epoch=29533 Loss=2.4809048175811768 Acc=0.741825280624695
Epoch=29534 Loss=2.4919559955596924 Acc=0.7403611517813568
Epoch=29535 Loss=2.484189510345459 Acc=0.7354807223035628
Epoch=29536 Loss=2.50373911857605 Acc=0.7354807223035628
Epoch=29537 Loss=2.4379899501800537 Acc=0.7364568081991215
Epoch=29538 Loss=2.4405179023742676 Acc=0.7379209370424598
Epoch=29539 Loss=2.4600820541381836 Acc=0.7364568081991215
Epoch=29540 Loss=2.484783172607422 Acc=0.7354807223035628
Epoch=29541 Loss=2.478447437286377 Acc=0.7364568081991215
Epoch=29542 Loss=2.504063129425049 Acc=0.7359687652513421
Epoch=29543 Loss=2.475508451461792 Acc=0.7340165934602245
Epoch=29544 Loss=2.4998390674591064 Acc=0.7315763787213275
Epoch=29

Epoch=29696 Loss=2.5650124549865723 Acc=0.6959492435334309
Epoch=29697 Loss=2.514803171157837 Acc=0.7076622742801366
Epoch=29698 Loss=2.5783145427703857 Acc=0.7179111761835042
Epoch=29699 Loss=2.513274669647217 Acc=0.7223035627135188
Epoch=29700 Loss=2.5141420364379883 Acc=0.7384089799902391
Epoch=29701 Loss=2.4988508224487305 Acc=0.7388970229380185
Epoch=29702 Loss=2.545685291290283 Acc=0.7388970229380185
Epoch=29703 Loss=2.480579137802124 Acc=0.7354807223035628
Epoch=29704 Loss=2.483826160430908 Acc=0.7291361639824304
Epoch=29705 Loss=2.522156000137329 Acc=0.7198633479746218
Epoch=29706 Loss=2.4982872009277344 Acc=0.7174231332357247
Epoch=29707 Loss=2.534299612045288 Acc=0.7125427037579307
Epoch=29708 Loss=2.502274751663208 Acc=0.7125427037579307
Epoch=29709 Loss=2.4854166507720947 Acc=0.714006832601269
Epoch=29710 Loss=2.479670524597168 Acc=0.7198633479746218
Epoch=29711 Loss=2.4815423488616943 Acc=0.7203513909224012
Epoch=29712 Loss=2.43241548538208 Acc=0.7227916056612982
Epoch=297

Epoch=29864 Loss=2.489837408065796 Acc=0.7447535383113714
Epoch=29865 Loss=2.453927755355835 Acc=0.7491459248413861
Epoch=29866 Loss=2.4852731227874756 Acc=0.7462176671547096
Epoch=29867 Loss=2.4748473167419434 Acc=0.7432894094680332
Epoch=29868 Loss=2.4436793327331543 Acc=0.7349926793557833
Epoch=29869 Loss=2.4412007331848145 Acc=0.7388970229380185
Epoch=29870 Loss=2.443748712539673 Acc=0.7388970229380185
Epoch=29871 Loss=2.402763605117798 Acc=0.7369448511469009
Epoch=29872 Loss=2.4975380897521973 Acc=0.7403611517813568
Epoch=29873 Loss=2.493014335632324 Acc=0.7384089799902391
Epoch=29874 Loss=2.4776387214660645 Acc=0.7413372376769156
Epoch=29875 Loss=2.4807510375976562 Acc=0.744265495363592
Epoch=29876 Loss=2.460874557495117 Acc=0.7403611517813568
Epoch=29877 Loss=2.4823122024536133 Acc=0.7374328940946804
Epoch=29878 Loss=2.4636359214782715 Acc=0.7359687652513421
Epoch=29879 Loss=2.476762294769287 Acc=0.7325524646168863
Epoch=29880 Loss=2.4798665046691895 Acc=0.7315763787213275
Epoch

In [5]:
correct_blocks = []
correct_block = dgl.sampling.sample_neighbors(g, infer_nids, -1)
correct_block = dgl.to_block(correct_block, infer_nids)
correct_blocks.insert(0, correct_block)

for _ in range(num_layers - 1):
    seeds = correct_blocks[0].srcdata[dgl.NID]
    correct_block = dgl.sampling.sample_neighbors(g, seeds, -1)
    correct_block = dgl.to_block(correct_block, seeds)
    correct_blocks.insert(0, correct_block)

with torch.no_grad():
    input_feats = g.ndata["features"][correct_blocks[0].srcdata[dgl.NID]]
    h = input_feats
    for layer_idx in range(num_layers):
        h = model.layer_foward(layer_idx, correct_blocks[layer_idx], h)

correct_acc = cal_acc(h.cpu().numpy(), g.ndata["labels"][infer_nids].cpu().numpy())

In [6]:
exclude_edges = g.out_edges(infer_nids, 'eid')

pe_blocks = []
pe_block = dgl.sampling.sample_neighbors(g, train_nids, -1, exclude_edges=exclude_edges)
pe_block = dgl.to_block(pe_block, train_nids)
pe_blocks.insert(0, pe_block)

for _ in range(num_layers - 1):
    seeds = pe_blocks[0].srcdata[dgl.NID]
    pe_block = dgl.sampling.sample_neighbors(g, seeds, -1, exclude_edges=exclude_edges)
    pe_block = dgl.to_block(pe_block, seeds)
    pe_blocks.insert(0, pe_block)

pes = []

with torch.no_grad():
    input_feats = g.ndata["features"][pe_blocks[0].srcdata[dgl.NID]]
    h = input_feats
    for layer_idx in range(num_layers):
        h = model.layer_foward(layer_idx, pe_blocks[layer_idx], h)
        if layer_idx != num_layers - 1:
            pes.append(h[:len(train_nids)])

assert torch.all(torch.tensor(train_nids).to(device) == pe_blocks[0].srcdata[dgl.NID][:len(train_nids)])

In [7]:
compute_with_pe_block = dgl.sampling.sample_neighbors(g, infer_nids, -1)
compute_with_pe_block = dgl.to_block(compute_with_pe_block, infer_nids)

with torch.no_grad():
    input_feats = g.ndata["features"][compute_with_pe_block.srcdata[dgl.NID]]
    h = input_feats
    for layer_idx in range(num_layers):
        h = model.layer_foward(layer_idx, compute_with_pe_block, h)
        if layer_idx != num_layers - 1:
            h = torch.concat((h, pes[layer_idx][compute_with_pe_block.srcdata[dgl.NID][h.shape[0]:]]))

approx_acc = cal_acc(h.cpu().numpy(), g.ndata["labels"][infer_nids].cpu().numpy())

In [8]:
print(f"Total accuracy={acc}")
print(f"Inference nodes correct accuracy={correct_acc}")
print(f"Inference nodes with PE accuracy={approx_acc}")

Total accuracy=0.7227916056612982
Inference nodes correct accuracy=0.7568359375
Inference nodes with PE accuracy=0.3857421875


In [9]:
# Sanity check

exact_pe_blocks = []
exact_pe_block = dgl.sampling.sample_neighbors(g, train_nids, -1)
exact_pe_block = dgl.to_block(exact_pe_block, train_nids)
exact_pe_blocks.insert(0, exact_pe_block)

for _ in range(num_layers - 1):
    seeds = exact_pe_blocks[0].srcdata[dgl.NID]
    exact_pe_block = dgl.sampling.sample_neighbors(g, seeds, -1)
    exact_pe_block = dgl.to_block(exact_pe_block, seeds)
    exact_pe_blocks.insert(0, exact_pe_block)

exact_pes = []

with torch.no_grad():
    input_feats = g.ndata["features"][exact_pe_blocks[0].srcdata[dgl.NID]]
    h = input_feats
    for layer_idx in range(num_layers):
        h = model.layer_foward(layer_idx, exact_pe_blocks[layer_idx], h)
        if layer_idx != num_layers - 1:
            exact_pes.append(h[:len(train_nids)])

assert torch.all(torch.tensor(train_nids).to(device) == pe_blocks[0].srcdata[dgl.NID][:len(train_nids)])

with torch.no_grad():
    input_feats = g.ndata["features"][compute_with_pe_block.srcdata[dgl.NID]]
    h = input_feats
    for layer_idx in range(num_layers):
        h = model.layer_foward(layer_idx, compute_with_pe_block, h)
        if layer_idx != num_layers - 1:
            h = torch.concat((h, exact_pes[layer_idx][compute_with_pe_block.srcdata[dgl.NID][h.shape[0]:]]))

acc_with_exact_pes = cal_acc(h.cpu().numpy(), g.ndata["labels"][infer_nids].cpu().numpy())

assert correct_acc == acc_with_exact_pes